# Final Project

In [2]:
# Import all required libraries
from __future__ import division # For python 2.*

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

np.random.seed(0)
%matplotlib inline
# Data Loading
X = np.genfromtxt('uci-cs178-f21/X_train.txt', delimiter=',')
Y = np.genfromtxt('uci-cs178-f21/Y_train.txt', delimiter=None)

# The test data
Xte = np.genfromtxt('uci-cs178-f21/X_test.txt', delimiter=',')

print("Shape of X: {}, Xte: {}, Y: {}".format(X.shape, Xte.shape, Y.shape))

Shape of X: (7423, 107), Xte: (7422, 107), Y: (7423,)
1.0


In [5]:
Xtr,Xva,Ytr,Yva = ml.splitData(X,Y,0.5)

knn = ml.knn.knnClassify(Xtr, Ytr)

print(knn.auc(Xtr, Ytr))
print(knn.auc(Xva, Yva))

1.0
0.522992028539224


In [7]:
Yhat = knn.predictSoft(Xte)
print(Yhat)
#Yte = np.vstack((np.arange(len(Yhat)), Yhat[:,1])).T


[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [9]:
Yte = np.vstack((np.arange(len(Yhat)), Yhat[:,1])).T
np.savetxt('f1_knn.txt',Yte,'%d, %.2f',header='Id,Predicted',delimiter=',')